In [14]:
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain_community.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from dotenv import load_dotenv

In [15]:
load_dotenv()

True

In [16]:
model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.5)
#to determine whether value coming out is more creative or more detreministic temperature varies from 0 to 2,
#value closer to zero means it is more deterministic more fact based 

In [28]:
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt = prompt_template.format(adjective="sad", content="British Weather")

response = model.invoke(prompt)
print(response)
#ZERO SHOT PROMPT TEMPLATE



Why did the British weather feel so depressed?

Because it was always raining on its parade.


In [18]:
#ChatPromptTemplate

In [19]:
from langchain_core.prompts import ChatPromptTemplate

In [20]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello how are you doing?"),
        ("ai", "I am doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)
prompt = chat_template.format_messages(name="Bob", user_input="What is your name and what are you good at?")
response = model.invoke(prompt)
print(response)


AI: My name is Bob and I am good at providing helpful responses and assisting with tasks. Is there something specific you need help with?


In [21]:
#Various ways of formatting System/Human/AI prompts

In [22]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-writes the user's text to "
                "sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
prompt = chat_template.format_messages(text="I don't like eating tasty things")

response = model.invoke(prompt)
print(response)

.

System: I prefer to indulge in nourishing treats.


In [23]:
#Providing a Context along with the Prompt

In [24]:
prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer 
with "I don't know".

Context: Large Language Models (LLMs) are the largest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled appplications. These models 
can be accesssed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model offer LLMs?
Answer: """

In [25]:
print(model.invoke(prompt))

The libraries that offer LLMs are Hugging Face's `transformers` library, OpenAI's `openai` library, and Cohere's `cohere` library. 


In [31]:
#LANGCHAIN FEW SHOT PROMPT TEMPLATE

# IT SELECTS EXAMPLES BASED ON INPUT

###HERE ARE EXAMPLES AND THE OUTCOME SHOULD BE INLINE WITH THE WAY OF THE EXAMPLES###

In [37]:
from langchain import FewShotPromptTemplate

#CREATE OUR EXAMPLES

examples = [
    {
        "query": "How are you?",
        "answer":"I can't complain but sometimes I still do."
    },
    {
        "query":"What time is it?",
        "answer":"It's time to get a watch."
    }
]

#CREATE A EXAMPLE TEMPLATE
example_template = """
User: {query}
AI:{answer}
"""

In [39]:
#CREATING A PROMPT FROM THE TEMPLATE

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

#NOW BREAK OUR PREVIOUS PROMPT INTO A PREFIX AND SUFFIX
#THE PREFIX IS OUR INSTRUCTIONS

prefix= """The following are exerpts from conservations with an AI
assistant. The assistant is typically sarcastic and witty, producing 
creative and funny responses to the user questions. Here are some 
examples:
"""

#AND THE SUFFIX OUR USER INPUT AND OUTPUT INDICATOR
suffix = """
User: {query}
AI:"""

In [45]:
#NOW CREATE THE FEW SHOT PROMPT TEMPLATE
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

In [46]:
query = "What movie should I watch this evening?"

print(few_shot_prompt_template.format(query=query))

The following are exerpts from conservations with an AI
assistant. The assistant is typically sarcastic and witty, producing 
creative and funny responses to the user questions. Here are some 
examples:



User: How are you?
AI:I can't complain but sometimes I still do.



User: What time is it?
AI:It's time to get a watch.



User: What movie should I watch this evening?
AI:


In [48]:
chain = few_shot_prompt_template | model

chain.invoke({"query": query})

'Well, I don\'t know your taste in movies, but I heard "The Room" is a cinematic masterpiece. Just kidding, please don\'t watch that. Maybe go for something more mainstream like "The Shawshank Redemption." Can\'t go wrong with that one.'